In [1]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation


# Training utils
import utils_xgboost

# Optuna
import optuna

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

c:\Users\Fer\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Cargamos los datos

In [2]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates = ['FECHAHORA'])

# Variables de entrenamiento

In [5]:
estacion = 4

variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5']

dependent = ['AQI_MP2_5']

number_of_features = len(variables)

training_days = 7 
forecast_days = 1/2
samples_per_day = 288
step = 288/2

# Creamos una variable que nos diga con cuantos meses de entrenamiento queremos contar para el X_train
train_months = relativedelta(months = 12)

input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test



# Procesamiento previo a entrenamiento

In [6]:
%%time

X_train, y_train, X_test, y_test = utils_xgboost.get_everything(datos, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)

CPU times: total: 13.5 s
Wall time: 13.5 s


In [7]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


(718, 2025)
(718, 144)
(168, 2025)
(168, 144)


# Prueba antes de usar optuna

In [8]:
%%time

params = {'learning_rate' : 0.025,
          'n_estimators' : 250,
          'max_depth': 2,
          'min_child_weight' : 1,
          'gamma': 0.0,
          'subsample': 0.98,
          'colsample_bytree': 0.98,
          'scale_pos_weight': 0.8,
          'seed': 42,
          'verbosity' : 0}


xgb_model = xgb.XGBRegressor(** params)

trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

prediction = trained_xgb_model.predict(X_test)
print('prediction', prediction.shape)
print('test', y_test.shape)

MSE = np.mean((prediction - y_test)**2)
print('RMSE: ', MSE**0.5)
MAE = np.mean(np.abs(prediction-y_test))
print('MAE: ', MAE)

prediction (333, 288)
test (333, 288)
RMSE:  9.781115562652332
MAE:  6.694983114898304
CPU times: total: 1h 56min 35s
Wall time: 10min 16s


In [8]:
%%time

def objective(trial):
    
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 10, 300),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
    }
    
    optuna_xgb_model = xgb.XGBRegressor(** params)
    
    trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

    prediction = trained_xgb_model.predict(X_test)

    MSE = np.mean((prediction - y_test)**2)
    RMSE = MSE**0.5
    print('RMSE: ', RMSE)
    MAE = np.mean(np.abs(prediction - y_test))
    print('MAE: ', MAE)

    MAPE = mean_absolute_percentage_error(prediction, y_test)
    
    return MAPE

study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials = 100)

trial = study.best_trial

[I 2023-03-11 17:50:28,960] A new study created in memory with name: no-name-a6e92982-d8c2-4fe2-b574-99d8e48ae62d
[I 2023-03-11 17:55:02,944] Trial 0 finished with value: 0.23865354376066467 and parameters: {'max_depth': 8, 'learning_rate': 0.05502064924079962, 'n_estimators': 167, 'min_child_weight': 6, 'gamma': 3.107205484822773e-05, 'subsample': 0.6202468633537521, 'colsample_bytree': 0.011144371186766578}. Best is trial 0 with value: 0.23865354376066467.


RMSE:  8.739763175424889
MAE:  5.8051675093513


[I 2023-03-11 17:55:46,101] Trial 1 finished with value: 0.3048953077973896 and parameters: {'max_depth': 3, 'learning_rate': 0.15937450562023958, 'n_estimators': 31, 'min_child_weight': 4, 'gamma': 0.009304173655425414, 'subsample': 0.016813293222987014, 'colsample_bytree': 0.15113137936283447}. Best is trial 0 with value: 0.23865354376066467.


RMSE:  10.43396125379585
MAE:  7.527313955039496


[I 2023-03-11 17:57:09,895] Trial 2 finished with value: 2.0234424705420975 and parameters: {'max_depth': 2, 'learning_rate': 0.8216178147141048, 'n_estimators': 73, 'min_child_weight': 9, 'gamma': 0.00012593432242008764, 'subsample': 0.02002317979457144, 'colsample_bytree': 0.07880630686326365}. Best is trial 0 with value: 0.23865354376066467.


...

RMSE:  10.587187495391927
MAE:  7.729405634814785


[I 2023-03-11 21:42:23,715] Trial 98 finished with value: 0.16759752998699923 and parameters: {'max_depth': 2, 'learning_rate': 0.03948126759605809, 'n_estimators': 281, 'min_child_weight': 3, 'gamma': 0.01850491918530355, 'subsample': 0.8012276321960156, 'colsample_bytree': 0.48413574957854655}. Best is trial 33 with value: 0.16667551561316735.


RMSE:  5.625224220596265
MAE:  3.6247588234051826


[I 2023-03-11 21:44:21,646] Trial 99 finished with value: 0.16737440410478882 and parameters: {'max_depth': 2, 'learning_rate': 0.039412508273739905, 'n_estimators': 280, 'min_child_weight': 3, 'gamma': 0.00521410921696577, 'subsample': 0.8087674110239372, 'colsample_bytree': 0.24203893286192182}. Best is trial 33 with value: 0.16667551561316735.


RMSE:  5.650478349516389
MAE:  3.632331419814989
CPU times: total: 1d 17h 18min 25s
Wall time: 3h 53min 52s


In [9]:
import joblib

joblib.dump(study, "optuna_studies/study_XGBOOST_230311_solo_mp_e4_7antes_1prediccion_cada12h_MAPE.pkl")
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

    max_depth: 2
    learning_rate: 0.036926703008863505
    n_estimators: 279
    min_child_weight: 2
    gamma: 0.007916443805057008
    subsample: 0.8545372011399259
    colsample_bytree: 0.76808354005096
